### Student name : Yogesh Haresh Bojja
### Student number : s3789918
### Course project number : Group 66

We have imported necessary libraries, dataset from the csv file is imported in the variable 'diamonds'. Imported dataset is also displayed below.

In [2]:
# Importing libraries
import pandas as pd
import numpy as np

In [3]:
diamonds = pd.read_csv('THA_diamonds.csv')

In [4]:
diamonds.head(5)

,cut,color,depth,price,carat
0,Good,D,63.6,low,0.44
1,Fair,F,64.2,low,0.45
2,Good,I,60.4,low,0.50
3,Good,F,56.8,low,0.45
4,Fair,F,64.3,low,0.45


### Part A.

We need to discretize the features 'carat' and 'depth' by equal-frequency binning technique. Hence we will apply quantile cut(qcut) with 3 bins for these features. we will bin both the features into 'category_1', 'category_2' and 'category_3' respectively.

In [5]:
diamonds['carat'] = pd.qcut(diamonds['carat'], q=3, 
                                     labels=['category_1', 'category_2', 'category_3'])
diamonds['depth'] = pd.qcut(diamonds['depth'], q=3, 
                                     labels=['category_1', 'category_2', 'category_3'])

After binning, dataset looks as follows.

In [6]:
diamonds.head(10)

,cut,color,depth,price,carat
0,Good,D,category_2,low,category_1
1,Fair,F,category_3,low,category_1
2,Good,I,category_1,low,category_1
3,Good,F,category_1,low,category_1
4,Fair,F,category_3,low,category_1
5,Fair,F,category_3,low,category_1
6,Good,D,category_2,low,category_1
7,Good,D,category_2,low,category_1
8,Good,D,category_2,low,category_1
9,Fair,F,category_3,low,category_1


### Part B.

We will be finding impurity for feature 'price' by 'entropy' split_criterion. Higher the value of entropy, higher is the impurity.

In [7]:
probs = diamonds['price'].value_counts(normalize=True)
entropy = -1 * np.sum(np.log2(probs) * probs)
entropy

1.7160130346557048

### Part C.

'Compute_impurity' calculates and returns value of impurity based on the 'impurity_criterion' we pass. 'Comp_feature_information_gain' takes dataframe, name of target feature, name of descriptive feature and split criterion as the parameters, it calculates information gain of the descriptive feature and returns it. 

In [8]:
def compute_impurity(feature, impurity_criterion):
    probs = feature.value_counts(normalize=True)
    
    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(np.log2(probs) * probs)
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))

def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
    
    # Impurity of the target feature 'price'
    target_entropy = compute_impurity(df[target], split_criterion)
    
    
    entropy_list = list()
    weight_list = list()
    level_list = list()
    
    # Find Impurity of all level from descriptive feature
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))
        level_list.append(level)

    print('Level list:', level_list)
    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    # summation of all Impurities of all the levels multiplied by its respective probability in descriptive feature
    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    # find information gain of the descriptive feature
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)
    
    print('====================')

    return(information_gain)

We need to find optimal root node of the decision tree. We could use gini index or entropy to find the information gain of the feature. We will be observing selection of the root node by entropy method. More the feature is homogenious, more is its information gain. Feature with highest information gain is selected as root node. 

In [9]:
split_criterion = 'entropy'
for feature in diamonds.drop(columns='price').columns:
    feature_info_gain = comp_feature_information_gain(diamonds, 'price', feature, split_criterion)

target feature: price
descriptive_feature: cut
split criterion: entropy
Level list: ['Good', 'Fair']
impurity of partitions: [1.68, 1.78]
weights of partitions: [0.717, 0.283]
remaining impurity: 1.7083
information gain: 0.00770000000000004
target feature: price
descriptive_feature: color
split criterion: entropy
Level list: ['D', 'F', 'I']
impurity of partitions: [1.657, 1.445, 1.833]
weights of partitions: [0.269, 0.434, 0.297]
remaining impurity: 1.617264
information gain: 0.09873599999999993
target feature: price
descriptive_feature: depth
split criterion: entropy
Level list: ['category_2', 'category_3', 'category_1']
impurity of partitions: [1.517, 1.749, 1.74]
weights of partitions: [0.349, 0.316, 0.335]
remaining impurity: 1.6650170000000002
information gain: 0.05098299999999978
target feature: price
descriptive_feature: carat
split criterion: entropy
Level list: ['category_1', 'category_2', 'category_3']
impurity of partitions: [-0.0, 1.365, 1.529]
weights of partitions: [0.335

By entropy method 'carat' has highest information gain, hence it proves to be optimal root node.

In [10]:
df_splits = pd.DataFrame(columns=['split', 'remainder', 'info_gain', 'is_optimal'])
df_splits.loc[len(df_splits)] = ["cut", 1.7083 , 0.00770, False]
df_splits.loc[len(df_splits)] = ["color", 1.617264 , 0.09873, False]
df_splits.loc[len(df_splits)] = ["depth", 1.66501 , 0.05098, False]
df_splits.loc[len(df_splits)] = ["carat", 0.95561 , 0.76038, True]

df_splits

,split,remainder,info_gain,is_optimal
0,cut,1.708300,0.00770,False
1,color,1.617264,0.09873,False
2,depth,1.665010,0.05098,False
3,carat,0.955610,0.76038,True


### Part D.

We have been told to assume 'carat' as the root node. Feature 'carat' has 3 levels i.e. 'category_1', 'category_2' and 'category_3', hence we need to subset out dataset by these 3 levels. We will calculate probability in 'price' column for all these respective levels. 

In [11]:
for level in diamonds['carat'].unique():
    df = diamonds[diamonds['carat']==level]
    probs = df['price'].value_counts(normalize=True)
    print("--------- ",level," ---------")
    print(probs)
        

---------  category_1  ---------
low    1.0
Name: price, dtype: float64
---------  category_2  ---------
medium     0.607595
low        0.278481
high       0.101266
premium    0.012658
Name: price, dtype: float64
---------  category_3  ---------
medium     0.419355
high       0.370968
premium    0.209677
Name: price, dtype: float64


We have populated the following table with the above results. Referring to the table below, first leaf condition is 'carat==category_1' where all the rows has its price equal to 'low' i.e. low_price_prob = 1.0, hence leaf prediction for this branch is 'price=low'. Second leaf condition is 'carat==category_2' where 60.7% of the rows have its price equal to medium, hence leaf_prediction is equal to 'price=medium'. In third leaf condition of 'carat==category_3' the probability of price being medium is most i.e. 0.419, hence leaf_prediction is set as 'price=medium'. 

In [12]:
df_pred = pd.DataFrame(columns=['leaf_condition', 'low_price_prob', 'medium_price_prob', 'high_price_prob',
                                'premium_price_prob', 'leaf_prediction'])
df_pred.loc[len(df_pred)] = ["carat==category_1", 1.0 , 0.0, 0.0, 0.0, "price=low"]
df_pred.loc[len(df_pred)] = ["carat==category_2", 0.278481 , 0.607595, 0.101266, 0.012658, "price=medium"]
df_pred.loc[len(df_pred)] = ["carat==category_3", 0.0 , 0.419355, 0.370968, 0.209677, "price=medium"]

df_pred

,leaf_condition,low_price_prob,medium_price_prob,high_price_prob,premium_price_prob,leaf_prediction
0,carat==category_1,1.000000,0.000000,0.000000,0.000000,price=low
1,carat==category_2,0.278481,0.607595,0.101266,0.012658,price=medium
2,carat==category_3,0.000000,0.419355,0.370968,0.209677,price=medium
